In [1]:
import pandas as pd
from selenium import webdriver
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [ ]:
#Patent들의 url을 Crawling 하는 부분
#selenuim을 사용한다.

urls = []

for j in range(10):
    if j == 0:
        print('Crawling 시작')
    driver = webdriver.Chrome('chromedriver.exe')
    url_page = str(j)
    driver.get(url='https://patents.google.com/?q=((solar+cell))&q=(+H02J3%2f381+)&before=priority:20101231&after=priority:20100101&language=ENGLISH&type=PATENT&num=100&page='+url_page)
    
    driver.implicitly_wait(time_to_wait=5)
    info = driver.find_element_by_xpath('//*[@id="resultsContainer"]')
    contents = info.find_elements_by_tag_name('search-result-item')
    for i in range(len(contents)):
        address = contents[i].find_element_by_tag_name('state-modifier').get_attribute('data-result') # patent/name/en
        url = 'patents.google.com/%s' % address
        urls.append('http://'+url)
    if j == 10:
        print('Crawling 끝')

df = pd.DataFrame(urls,columns=['url'])
df.to_csv('url.csv')
print('crawling한 url주소를 url.csv 라는 이름으로 저장함')
print(len(df))

df_list = list(df['url'])
len(df_list)

In [ ]:
# 위에서 crawling한 url을 이용해서 urlopen으로 title , abstract를 crawling 하는 code
# 가끔 title이나 abstract가 없는 경우가있다. 그런 경우에는 No title(description)을 저장한다.

title = []
abstract = []
label_all = []
meta_label = []

for k in df_list:
    print(k+' Crawling중입니다.')
    html = urlopen(k)
    bsObject = BeautifulSoup(html,'html.parser')
    
#    print('제목시작')
    if len(bsObject.find_all('title')) == 0:
        title.append('No title')
        print("없음")
    else:  
        for a in bsObject.find_all('title'):
            text = a.get_text()
            print("Title : ",len(text))
            title.append(text)
#    print('제목 끝')
    
#    print('abstract 시작')
    if len(bsObject.find_all('meta',{'name':'description'})) == 0:
        abstract.append('No description')
        print('abstract 없음')
    else:
        for b in bsObject.find_all('meta',{'name':'description'}):
            b = str(b)
            b =b.replace('<meta content="\n','')
            c = b.replace('name="description"/>','')
            print("Abstract : ",len(c))
            abstract.append(c)

#     print('abstract 끝')

title_df = pd.DataFrame(title,columns=['Title'])
abstract_df = pd.DataFrame(abstract,columns=['Abstract'])

patent_crawled_df = pd.concat([title_df,abstract_df],axis=1)

patent_crawled_df.to_csv("Final_H01L_31_crawling.csv")